In [32]:
from pyproj import Proj
from shapely.geometry import Point, Polygon, LineString, MultiPolygon


#pr = Proj('+init=EPSG:27700')

# This is a stereographic projection which uses Finsbury Square as origin
pr = Proj('+proj=stere +lat_0=51.5209993 +lon_0=-0.0863623 +k=1 +x_0=0 +y_0=0 +a=6371000 +b=6371000 +units=m +no_defs')

file_name="data/pubs.tsv"
pubs = dict([(int(x[0]), {
                'name': x[4], 
                'o': (float(x[1]), float(x[2])),
                'p': pr(float(x[1]), float(x[2])),
                'point': Point(float(x[1]), float(x[2])),
                'id': int(x[0])}) 
             for x in [line.strip().split('\t') 
                       for line in file(file_name).readlines()] if len(x)>4])
len(pubs)
pubs[1142096055]

{'id': 1142096055,
 'name': "Finch's",
 'o': (-0.0872945, 51.5217532),
 'p': (-64.49651771904955, 83.83026593644367),
 'point': <shapely.geometry.point.Point at 0x1118cdd90>}

In [12]:
from shapely.geometry import asShape, mapping
import json

wards = json.load(file('data/wards.json'))['features']


{u'geometry': {u'coordinates': [[[-0.330679062942453, 51.329011010602905],
    [-0.330594448736231, 51.329088039878236],
    [-0.330506117851447, 51.329148829506465],
    [-0.330409211490547, 51.32920859757066],
    [-0.330292121849854, 51.32927077559537],
    [-0.329913461974029, 51.32946051543075],
    [-0.329633594619526, 51.32965705855314],
    [-0.329428784834638, 51.32984208201042],
    [-0.329320915687227, 51.32996643925324],
    [-0.32921448062474, 51.330090816854394],
    [-0.32911632355624, 51.33026387227744],
    [-0.329112955750983, 51.3303960148678],
    [-0.329171167711951, 51.33072957000635],
    [-0.329191302316349, 51.33096546228584],
    [-0.329302910546211, 51.331528189994685],
    [-0.329362771534731, 51.33161896942137],
    [-0.329554552333886, 51.33187169784059],
    [-0.329578988615119, 51.33218678585108],
    [-0.329594210640675, 51.33228142478945],
    [-0.329690000087309, 51.332489619653416],
    [-0.32971930401878, 51.332670787926034],
    [-0.329718711471269

In [20]:
for ward in wards[0:1]:
    ward_shape = asShape(ward['geometry'])
    count = 0
    for pub in pubs.values():
        if ward_shape.contains(pub['point']):
            count += 1
    print ward['properties']['NAME'], count

Chessington South 3


In [22]:
from rtree import index
tree = index.Index()
for pub in pubs.values():
    tree.insert(pub['id'], (pub['o'][0], pub['o'][1],pub['o'][0], pub['o'][1]))
tree

In [23]:
asShape(wards[0]['geometry']).bounds

(-0.330756473741289, 51.32631963991207, -0.285101065668824, 51.36635219034912)

In [30]:
for ward in wards:
    ward_shape = asShape(ward['geometry'])
    count = 0
    for pub_id in tree.intersection(ward_shape.bounds):
        pub = pubs[pub_id]
        if ward_shape.contains(pub['point']):
            count += 1
    ward['properties']['pubcount'] = count
    ward['properties']['pubcount_per_hectare'] = count / ward['properties']['HECTARES']
    print ward['properties']['NAME'], count

Chessington South 3
Tolworth and Hook Rise 1
Berrylands 5
Alexandra 2
Beverley 2
Coombe Hill 1
Chessington North and Hook 2
Surbiton Hill 1
Old Malden 2
St. Mark's 12
Grove 16
Canbury 7
Norbiton 6
Coombe Vale 2
St. James 2
Tudor 3
Coulsdon East 2
Selsdon and Ballards 1
Coulsdon West 2
Waddon 3
Kenley 2
Purley 3
Sanderstead 0
Heathfield 6
Fairfield 24
Broad Green 7
West Thornton 5
Bensham Manor 2
Norbury 0
New Addington 0
Croham 2
Fieldway 0
Shirley 1
Selhurst 9
Ashburton 0
Woodside 2
Thornton Heath 1
Upper Norwood 5
South Norwood 7
Addiscombe 7
Darwin 7
Hayes and Coney Hall 2
Bromley Common and Keston 5
Chelsfield and Pratts Bottom 7
Biggin Hill 1
West Wickham 4
Clock House 2
Kelsey and Eden Park 3
Farnborough and Crofton 4
Shortlands 0
Bromley Town 14
Bickley 3
Petts Wood and Knoll 4
Crystal Palace 5
Penge and Cator 10
Copers Cope 10
Plaistow and Sundridge 8
Chislehurst 10
Mottingham and Chislehurst North 1
Orpington 1
Cray Valley West 0
Cray Valley East 3
Bedfont 3
Hanworth 2
Cranfor

In [31]:
feature_collection = {"type": "FeatureCollection",
                      "features": wards
                      }
with open('data/wards_with_pubs.json', 'w') as f:
    json.dump(feature_collection, f)